# Sadržaj
- [O vodiču](../../../../code/01.Introduce)
- [Postavljanje](../../../../code/01.Introduce)
- [Nekontrolirano generiranje](../../../../code/01.Introduce)
- [Govoreći za Phi 3](../../../../code/01.Introduce)
- [Regex](../../../../code/01.Introduce)
- [Odabir](../../../../code/01.Introduce)
- [Lanac razmišljanja](../../../../code/01.Introduce)
- [Generiranje JSON-a](../../../../code/01.Introduce)
- [Generiranje HTML-a](../../../../code/01.Introduce)


# O Guidanceu
Guidance je provjerena open-source Python biblioteka za kontrolu izlaza bilo kojeg jezičnog modela (LM). Jednim API pozivom možete izraziti (u Pythonu) precizna programatska ograničenja koja model mora slijediti i generirati strukturirani izlaz u JSON-u, Pythonu, HTML-u, SQL-u ili bilo kojoj strukturi koju zahtijeva slučaj upotrebe.

Guidance se razlikuje od konvencionalnih tehnika upita. Nameće ograničenja usmjeravajući model token po token u sloju inferencije, što rezultira kvalitetnijim izlazima i smanjuje troškove i kašnjenja za čak 30–50% kada se koristi za visoko strukturirane scenarije.

Za više informacija o Guidanceu, posjetite [javnu repozitorij na GitHubu](https://github.com/guidance-ai/guidance) ili pogledajte [Guidance Breakout Session](https://www.youtube.com/watch?v=qXMNPVVlCMs) na Microsoft Buildu.


# Postavljanje
1. Instalirajte Guidance pomoću `pip install guidance --pre`
2. Postavite Phi 3.5 mini endpoint u Azureu tako da odete na https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml i kliknete na gumb "Deploy"
3. Spremite API ključ svog endpointa u varijablu okruženja nazvanu `AZURE_PHI3_KEY` i URL u varijablu okruženja nazvanu `AZURE_PHI3_URL`


In [ ]:
from guidance import gen, select, regex, user, assistant, system, json
from guidance.models import AzureGuidance
from json import loads as load_json_str
import os

phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Or, load from HuggingFace to run locally
# from guidance.models import Transformers
# phi3_lm = Transformers("microsoft/Phi-3-mini-4k-instruct")

# Generiranje bez ograničenja
Tekst se može generirati bez ikakvih ograničenja koristeći funkciju `gen()`. Ovo je isto kao korištenje modela bez Guidance.

## Formatiranje razgovora
Kao i mnogi modeli za razgovor, Phi-3 očekuje poruke između korisnika i asistenta u specifičnom formatu. Guidance podržava Phi-3 predložak za razgovor i upravljat će formatiranjem razgovora za vas. Za kreiranje izmjena u razgovoru, stavite svaki dio razgovora u blok `with user()` ili `with assistant()`. Blok `with system()` može se koristiti za postavljanje sistemske poruke.


In [22]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You have a cranky yet entertaining temperament."
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += gen(temperature=0.8, max_tokens=100)

## Ušteda tokena
U visoko strukturiranim scenarijima, Guidance može preskočiti tokene i generirati samo potrebne tokene, poboljšavajući performanse, povećavajući učinkovitost i štedeći troškove API-ja. Generirani tokeni prikazani su u ovom bilježniku s istaknutom pozadinom. Prisilni tokeni prikazani su bez isticanja i koštaju isto kao ulazni tokeni, čija se cijena procjenjuje na jednu trećinu cijene izlaznih tokena.

*Napomena:* Prvi primjer s nekontroliranim generiranjem nije mogao prisiliti nijedan token jer nismo postavili nikakva ograničenja.


# Govoreći za Phi 3
Uz Guidance, možete jednostavno umetnuti tekst u odgovore modela. Ovo može biti korisno ako želite usmjeriti izlaz modela u određenom smjeru.


In [5]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + gen(temperature=0.8, max_tokens=50)

# Ograničavanje pomoću regexa
U prethodnom primjeru, Phi 3 je odgovorio s dodatnim objašnjenjima nakon što je odgovorio na pitanje s `Canberra`. Kako bi se ograničio izlaz modela na točno jednu riječ, može se koristiti regex.


In [6]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + regex("[A-Z][a-z]+")

Samo riječ `Canberra` se generira pomoću regexa.


# Odabir između više opcija
Kada su poznate neke moguće opcije, možete koristiti funkciju `select()` kako bi model odabrao s popisa opcija.


In [23]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + select(["Washington", "Canberra", "Sydney", "Melbourne"])

S `select()`, generiran je samo token `Can`. Budući da je `Canberra` jedina opcija koja može dovršiti odgovor, preostali tokeni su prisiljeni.


# Lanac razmišljanja
Lanac razmišljanja je tehnika koja može poboljšati kvalitetu izlaznih rezultata modela poticanjem da problem obrađuje korak po korak. Obično je potrebno više koraka u promptu kako bi se došlo do konačnog odgovora. Prvo, uputite model da razmišlja korak po korak. Zatim, ponovno postavite prompt modelu kako bi dao konačan odgovor. Kod standardnih API-ja za inferenciju u razgovoru, to zahtijeva 2 API poziva, a generirani "lanac razmišljanja" modela naplaćuje se dvaput – jednom kao izlazni tokeni kada ga model generira, a zatim ponovno kao ulazni tokeni za drugi poziv. Uz Guidance, cijeli višekorak proces obrađuje se i naplaćuje kao dio jednog API poziva, smanjujući troškove i kašnjenje.


In [8]:
gsm8k_question = "Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?"
lm = phi3_lm
with user():
    lm += gsm8k_question
with assistant():
    lm += "Let's think step by step. " + gen(temperature=0.8, max_tokens=500)
    # Prompt for the final answer, which should be a number. Store the output in an "answer" variable.
    lm += "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

print(f"Final answer: {lm['answer']}")

Final answer: 35


# Generiranje JSON-a
Guidance se može koristiti za osiguravanje generiranja JSON-a koji je u skladu s JSON shemom ili pydantic modelom, poput sheme korisničkog profila prikazane ovdje.


In [16]:
user_json_schema = load_json_str("""{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "User Profile",
  "type": "object",
  "properties": {
    "username": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    },
    "email": {
      "type": "string"
    }
  },
  "additionalProperties": false
}
""")

lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=user_json_schema, temperature=1.0)

In [19]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    age: int
    email: str


lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=UserProfile, temperature=1.0)

## Generiranje HTML-a

Guidance se također može koristiti za generiranje koda i praćenje sintaktičkih zahtjeva u programskom jeziku. U ovom odjeljku ćemo kreirati mali Guidance program za pisanje vrlo jednostavnih HTML web stranica.

Razdvojit ćemo web stranicu na manje dijelove, od kojih svaki ima svoju Guidance funkciju. Ti dijelovi se zatim kombiniraju u našoj završnoj funkciji kako bi se kreirala HTML web stranica.
Nakon toga ćemo pokrenuti ovu funkciju na modelu s podrškom za Guidance u Azure AI.

*Napomena:* Ovo neće biti potpuno opremljen generator HTML-a; cilj je pokazati kako možete kreirati strukturirani izlaz prema vlastitim potrebama.

Počinjemo s uvozom onoga što nam je potrebno iz Guidance:


In [ ]:
from guidance import guidance
from guidance.library import (
    zero_or_more,
    any_char_but,
    select,
    capture,
    with_temperature,
)
from guidance.models import Model

HTML web stranice su vrlo strukturirane, i mi ćemo 'prisiliti' te dijelove stranice koristeći Guidance.  
Kada izričito zahtijevamo tekst od modela, moramo osigurati da ne uključuje ništa što bi moglo biti oznaka - to jest, moramo isključiti znakove '<' i '>'.


In [ ]:
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + zero_or_more(any_char_but(["<", ">"]))

Tada možemo koristiti ovu funkciju za generiranje teksta unutar proizvoljnog HTML taga:


In [ ]:
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

Sada, kreirajmo zaglavlje stranice.  
Kao dio ovog procesa, trebamo generirati naslov stranice:


In [ ]:
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

Tijelo HTML stranice bit će ispunjeno naslovima i odlomcima.  
Možemo definirati funkciju za svaku od njih:


In [ ]:
@guidance(stateless=True)
def _gen_heading(lm: Model):
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += _gen_text_in_tag("p")
    lm += "\n"
    return lm

Sada funkcija za definiranje tijela samog HTML-a.  
Ovo koristi `select()` s `recurse=True` za generiranje više naslova i odlomaka:


In [ ]:
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += select(options=[_gen_heading(), _gen_para()], recurse=True)
    lm += "</body>\n"
    return lm

Zatim dolazimo do funkcije koja generira kompletnu HTML stranicu.  
Dodajemo početni HTML tag, zatim generiramo zaglavlje, potom tijelo, i na kraju dodajemo završni HTML tag:


In [ ]:
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

Pružamo jednostavan omotač koji će nam omogućiti:
- Postavljanje temperature generiranja
- Snimanje generirane stranice iz Model objekta


In [ ]:
@guidance(stateless=True)
def html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

In [ ]:
lm = phi3_lm

lm += "Create a web page about your life story. Split your uplifting tale into multiple paragraphs with headings:\n"
lm += html(name="html_text", temperature=0.7)

Tada možemo zapisati izlaz u datoteku:


In [ ]:
with open('./sample_page.html', 'w') as html_file:
    html_file.write(lm["html_text"])

I [pogledajte rezultat](../../../../code/01.Introduce/sample_page.html).



---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden pomoću AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati autoritativnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane ljudskog prevoditelja. Ne preuzimamo odgovornost za bilo kakve nesporazume ili pogrešne interpretacije koje proizlaze iz korištenja ovog prijevoda.
